# Coding A Basic Blockchain
## Josh Butikofer - Adobe (@jbutikofer - butikofe@adobe.com)

**DISCLAIMER:** _All opinions, ideas, and code expressed here are my own._

This is a live coding tutorial that will step you through the basic steps of building a _very_ basic blockchain written in Python. It was originally presented as a Live Coding Session at OSCON 2018 in Portland, OR. This notebook requires Python 3.7 or greater. See the accompanying `Pipfile` file for other dependencies.

## Definitions

Although there are several slightly different definitions for what constitutes a blockchain, this tutorial assumes the Wikipedia definition. Paraphrased from Wikipedia: A blockchain is composed of blocks that are cryptographically linked together to form a long chain. Because each block has a cryptographic hash of the previous block we can easily verify that the chain is immutable. Blocks also contain transactions and a timestamp. The transactions record some transfer of value between parties and typically use public-key cryptography to verify the transactions' integrity. Because value transfer by way of transactions is a major use case of blockchains, they are often called "distributed ledgers." (I like this moniker better than blockchain.) 

Blockchains derive a lot of their advantage by being decentralized in a peer-to-peer network. Because of this there needs to be a decentralized consensus algorithm so all the peers can agree on the current state of the chain that is distributed among them.

Put another way, a blockchain is a database that represents a ledger and has some of the following key characteristics:

* Cryptographically verifiable and auditable
* Immutable
* Implements a decentralized consensus algorithm
* Uses a peer-to-peer network to achieve decentralization

## Underlying Technology

An interesting observation is that the individual technologies used in blockchain are not new. The novelty of blockchain was was the combination of this technology to solve the problems inherent in digital cash systems--_without the need for a central authority._

Our basic blockchain will use the same cryptographic tech that mainstream ones do: hashing and public-key signatures. We will also use one of the most popular decentralized consensus algorithms, "Proof-of-Work", to show how blocks are made and shared in a distributed ledger.

## The Nuts and Bolts

Most blockchains are built from  a core set of nuts and bolts.

A blockchain has:

* **Transactions**: These are the entries in the ledger. In our example these will describe how user X wants to send some tokens to user Y. Transactions are cryptographically signed by user X to prove that they authorize the transfer of tokens.

* **Blocks**: Transactions are bundled up together into a block. Depending on the blockchain, there can be hundreds or thousands of transactions in a block. The transactions are validated and then the block includes the previous block's hash, linking itself to the previous block. There is also a process called "mining" or "forging" that proves to the network of blockchain participants that this block's creation can be trusted.

* **Blockchain**: All successfully mined blocks are strung together in a big, verifiable, and immutable chain. This is the system of record--the distributed ledger.

* **Nodes**: Computers around the world that participate in the distributed ledger network and are called "nodes." They often have a full copy of the chain and are responsible for accepting transactions, making new blocks, and updating the blockchain. They communicate via the other nodes in a P2P fashion. The decentralized consensus algorithm becomes critical here to help all nodes agree on what is the definitive chain since each node is working to grow the chain. *In our simple example, we will only have one node!*


# Enough talk--let's get coding!

## Transactions

First we will implement a **Transaction**. Transactions need to have:

* Sender address
* Recipient address
* Some payload--in our case this will be the amount of "tokens" that are being transferred.

**Another CRITICAL rule that satisifies one of the core tenets of blockchain: transactions need to be cryptographically signed. This verifies that the sender really wants to send tokens and someone is not stealing them!**

In [1]:
# We will be using some of the niceties of Python 3.7 and a helpful
# crypto library to help with public-key signing
from dataclasses import dataclass, field
from typing import *
import rsa # Don't implement your own
import hashlib
import time

In [12]:
@dataclass  # New Python 3.7 feature!
class Transaction:
    sender_address: rsa.PublicKey
    recipient_address: rsa.PublicKey
    value: float
    signature: bytes = field(default=b'')
       
    def sign(self, sender_private_key: rsa.PrivateKey) -> None:
        """Takes a private key, which the sender should never share, 
        and signs this transaction to verify they want to transfer the
        tokens. We sign the core attributes of the transaction using 
        the private key. We will use SHA-256 as the signature hashing 
        algorithm."""
        
        # sign and then validate to make sure the signature is valid
        
        self.signature = rsa.sign(self.get_header(), sender_private_key, 'SHA-256')
        self.validate()
        
    def get_header(self) -> bytes:
        """Provides the core info needed to sign the transaction--basically
        everything BUT the signature."""
        
        return (str(self.sender_address) + str(self.recipient_address) + str(self.value)).encode()
        
    def validate(self) -> None:
        """Validates the integrity of this transaction by using RSA to
        verify it is signed properly. Gets the core data about the 
        transaction, and the signature made using the private key,
        and uses the public key to validate it is signed. The public key
        is the sender_address."""
        
        rsa.verify(self.get_header(), self.signature, self.sender_address)
    


## Blocks

Now that we have transactions figured out (we think) we can focus on **Blocks**. Blocks needs to have:

* Block number (to make it easy to identify them)
* Timestamp
* A list of transactions included in the block
* The hash of the previous block in the chain (this is important for immutability)
* A hash of this block (again, to help enforce immutability)
* Nonce (this is used to help implement the distributed trust consensus algorithm)

The most important thing about a block is we must be able to tell if someone has changed the block and must prove that its addition to the chain is valid, hence the cryptographic hashing.

In [13]:
@dataclass
class Block:
    num: int
    timestamp: float
    prev_block_hash: str
    transactions: List[Transaction]
    block_hash: str = field(default="")
    nonce: int = field(default=0)
      
    def hash(self) -> str:
        """Uses SHA-256 to hash the header of the block (the core
        attributes of the block). Saves the hash to the block object
        for later use and also returns the hash in hex format."""
        self.block_hash = hashlib.sha256(self.get_header()).hexdigest()
        return self.block_hash
    
    def get_header(self) -> bytes:
        """Returns a string that represents the core attributes that
        uniquely identify the block AND link it to the previous block
        (forming the chain). These attributes include: block num,
        timestamp, previous block hash, transactions
        (real chains use a Merkle root hash for efficiency),
        and a nonce."""
        
        return (str(self.num) + str(self.timestamp) + str(self.prev_block_hash) + str(self.transactions) +
                str(self.nonce)).encode()
    
    def validate(self, proof_of_work_func: Callable[[str], bool]) -> bool:
        """Using the given function, ensure that this block hashes
        correctly, adhering to the agreed-upon consensus algorithm."""
        return proof_of_work_func(self.hash())

## Blockchain Node

Now that we have Transactions and Blocks, we need to have an object to hold the chain and put them all together! We will implement some code that acts as a "node" on the blockchain to accept transactions, mine blocks, and validate things.

A **Blockchain Node** needs to have the following:

* An address for it to receive reward tokens for mining new blocks!
* A authoritative list of blocks in the chain
* A way to receive new transactions (and validate them)
* A way to mine new blocks, thereby adding transactions into the chain
* A way to validate the chain, to prove that it adheres to the requirements of a distributed ledger and can be trusted

Let's do it!

In [40]:
REWARD_AMOUNT = 2.0

class BlockchainNode:
    
    def __init__(self, miner_address: rsa.PublicKey) -> None:
        self.miner_address: rsa.PublicKey = miner_address
        self.blocks: List[Block] = []
        self.pending_transactions: List[Transaction] = []
        self.proof_of_work_func: Callable[[str], bool] = lambda x: x.endswith('00000')
            
        self.mine_block()
      
    def submit_transaction(self, transaction: Transaction) -> None:        
        """This is used to submit a new transaction to the chain. End-users
        send transactions and aren't concerned about the blocks, per se.
        This function takes a signed transaction will validate it is
        cryptographically sound. It will then need to check that there
        is sufficient balance to make the transfer. If these are good,
        we will add the transaction to a list of those that will be
        considered when the next block is mined."""
        
        # Ensure the transaction is properly signed by the private key
        transaction.validate()
        
        # Make sure the funds exist for the requested transaction        
        balance = self.get_balance(transaction.sender_address)
        
        if balance < transaction.value:
            raise Exception("Insufficient funds!")
     
        # Transaction checks out--add to the list of our pending
        # transactions!
        
        self.pending_transactions.append(transaction)
        
    def mine_block(self) -> None:
        """This function bundles all pending transactions and mines a new
        block. Mining is the process by which a node creates a new block.
        This is where the decentralized consensus algorithm comes in.
        We will be using a proof-of-work algorithm that is computationally
        expensive. This prevents bad actor nodes from flooding the P2P
        network with invalid transactions, blocks, or chains. It is too
        expensive to rewrite history, making the blockchain more secure
        from these kinds of attacks. This function will mine
        the new block and then append it to the chain automatically.
        
        Nodes that successfully mine a new block get a token award.
        This is how new tokens start to circulate up in the chain and
        this is supposed to incentivize more peers on the network.
        """
        
        # Make sure we have a genesis block to start the chain
        if len(self.blocks) <= 0:
            prev_hash = "-"
            next_num = 0
        else:
            prev_block = self.blocks[-1]
            prev_hash = prev_block.block_hash
            next_num = prev_block.num + 1
        
        # Get the previous block in the chain and use its number
        # and hash to incorporate into the new block.
        
        new_block = Block(next_num, time.time(), prev_hash, self.pending_transactions.copy())
        
        # Add a reward transaction at the end of the block for this node
        
        reward = Transaction(None, self.miner_address, REWARD_AMOUNT)
        
        new_block.transactions.append(reward)
        
        # Execute proof-of-work
        self.execute_pow(new_block)
        
        # Add the block to the chain
        self.blocks.append(new_block)
        
        # Reset our pending transactions
        self.pending_transactions.clear()
        
        # Print out success
        print(f"Successfully mined new block {new_block.num}")
        
        

    def execute_pow(self, block: Block) -> None:
        """Using the defined proof-of-work lambda, increment the nonce 
        on the block until we satisfy the lambda's assertion. Since this
        iterating and hashing can take a long time and a lot of CPU,
        it can become computationally intensive. When this function
        returns the block will have a nonce and hash that meet the
        consensus algorithm requirements and can be very easily checked."""
        
        # Print out success
        block.nonce = 0
        while not block.validate(self.proof_of_work_func):
            block.nonce += 1
            
            
        print(f"Successfully mined new block {block.num} with nonce {block.nonce}")
        
        
    def validate_chain(self) -> None:
        """Verifies that this chain is cryptographically sound and
        has not been modified. Also ensures that all blocks meet the
        conensus algorithm requirements. This function is MUCH faster
        than mining new blocks: this is why the proof-of-work algorithm
        works well. Verification is very fast, but mining new blocks
        is difficult."""
        
        # Cycle through each block and compare previous block hashes
        # and verify proof-of-work compliance.
        
        prev_hash = "-"
        for b in self.blocks:
            if b.prev_block_hash != prev_hash:
                raise Exception("Previous hashes don't match")
            elif not b.validate(self.proof_of_work_func):
                raise Exception("Something is wrong with PoW hash")
                
            prev_hash = b.block_hash
        
        # Print out success!
        
        print(f"Successfully validated chain of size {len(self.blocks)}")
        
    def get_balance(self, address: rsa.PublicKey) -> float:
        """This method provides an easy way to traverse the chain to
        find out the balance of the given address. Bitcoin doesn't
        necessarily work this way, but other chains do, and it works
        fine for our purposes. This method will return the token balance
        of the given address, returning 0.0 if the address has never been
        seen on the chain before."""
        
        # Start balance at 0 and cycle through all blocks
        # and their transactions, looking for the given
        # address and keeping track of the balance accordingly.
        balance = 0.0
        for b in self.blocks:
            for t in b.transactions:
                if t.sender_address == address:
                    balance -= t.value
                elif t.recipient_address == address:
                    balance += t.value
                    
        return balance
            

            

Now we can try and create transactions and mine some blocks. First we need some RSA public keys to act as addresses and to allow for signing. We can use the included `rsa` module to create these for us. Due to the requirements of RSA signing with SHA-256, we will need at least 512-bit keys:

In [41]:
public_key, private_key = rsa.newkeys(512)
receiving_public_key, receiving_private_key = rsa.newkeys(512)

Now we can use our public and private keys to make our first transaction:

In [42]:
t1 = Transaction(public_key, receiving_public_key, 0.5)
t1.sign(private_key)

Let's make another one. It's easy!

In [43]:
t2 = Transaction(public_key, receiving_public_key, 1.0)
t2.sign(private_key)


What happens if we try to create a transaction to take tokens from an address for which we don't have the private key?

In [44]:
bad_t = Transaction(public_key, receiving_public_key, 1000.0)
bad_t.sign(private_key)

Now we are ready to submit our transactions to a blockchain node. Let's create one:

In [45]:
node = BlockchainNode(public_key)
node.submit_transaction(t1)

Successfully mined new block 0 with nonce 65479
Successfully mined new block 0


The transaction will not be included in the blockchain until a block is mined:

In [46]:
node.mine_block()

Successfully mined new block 1 with nonce 613329
Successfully mined new block 1


We can test our implementation, and validate the integrity of the chain, by using the `validate_chain()` method:

In [47]:
node.validate_chain()

Successfully validated chain of size 2


Let's try to hack the chain by changing the value of a transaction...the immutability and validity of transactions should be violated and the validation should fail!

In [33]:
hack = node.blocks[1].transactions[0]
orig_value = hack.value
hack.value = 100000.0
node.validate_chain()

Exception: Something is wrong with PoW hash

Restoring the transaction back to its original state will allow the chain to be valid again:

In [34]:
hack.value = orig_value
node.validate_chain()

Successfully validated chain of size 2


Now we can show how "spending checks" are enforced. Let's try submitting a transaction that has a larger amount than our balance:

In [39]:
#node.mine_block()
node.get_balance(public_key)
node.submit_transaction(t2)
node.mine_block()
node.get_balance(receiving_public_key)

Successfully mined new block 3 with nonce 17112
Successfully mined new block 3


1.5

Last, but not least, let's explore what happens if we increase the complexity of our PoW algorithm.